In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

In [9]:
num_samples = 100
embedding_size = 512
image_embeddings = np.random.randn(num_samples, embedding_size)
print(image_embeddings.shape)

cols = [
    'cod_modelo_color', 'des_filename','des_sex','des_age','des_fabric','des_product_type'
]
metadata_df = pd.read_csv('data/product_data.csv')[cols]
# One-hot encode all columns except the first
metadata_encoded = pd.get_dummies(metadata_df.iloc[:, 2:], drop_first=False)

# Combine the first column back with the encoded columns
metadata_encoded = pd.concat([metadata_df.iloc[:, [0]], metadata_encoded], axis=1)
print(metadata_encoded.values.shape)

(100, 512)
(61484, 63)


In [12]:
import pickle

# Path to the pickle file
pickle_file = 'embeddings/embeddings_fclip.pkl'

# Load the data from the pickle file
with open(pickle_file, 'rb') as file:
    data = pickle.load(file)
    
labels = pd.read_csv('transformed_attribute_data.csv')
# Create a new DataFrame with the transformed values
validity = labels.copy()
validity.iloc[:, 1:] = validity.iloc[:, 1:].applymap(lambda x: 0 if x == "INVALID" else 1)

# Display the new DataFrame
validity.head()

C:\Users\thomasg\AppData\Local\Temp\ipykernel_13904\834055054.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  validity.iloc[:, 1:] = validity.iloc[:, 1:].applymap(lambda x: 0 if x == "INVALID" else 1)


,cod_modelo_color,cane_height_type,closure_placement,heel_shape_type,knit_structure,length_type,neck_lapel_type,silhouette_type,sleeve_length_type,toecap_type,waist_type,woven_structure
0,81_1034451,0,0,0,0,0,1,1,1,0,0,0
1,81_1034525,0,0,0,0,0,1,1,1,0,0,0
2,81_1035318,0,0,0,0,1,0,0,0,0,0,0
3,81_1035321,0,0,0,0,1,0,0,0,0,0,0
4,81_1035361,0,0,0,0,1,1,0,0,0,0,0


In [19]:
# Ensure X and Y are mutable by converting them to lists for appending
X = []
Y = []

# Iterate over each key in the dictionary
for key, value in data.items():
    # Find the row in 'validity' where the first column matches the key
    matching_rows = validity[validity.iloc[:, 0].apply(lambda x: x in key)]
    
    # Extract all columns except the first one for the matching rows
    if not matching_rows.empty:
        for _, row in matching_rows.iterrows():
            Y.append(row.iloc[1:].values)
            X.append(value)

# Convert X and Y back to numpy arrays after appending
X = np.array(X)
y = np.array(Y)

In [55]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), test_size=0.2, random_state=42)
colnames = labels.columns[1:]

a = np.mean(y, axis=0)
a = np.max((1-a, a), axis=0)

for i in range(y_test.shape[1]):
    print("#####################")
    print(colnames[i])
    print("#####################")
    dummyacc = a[i]
    print("Dummy")
    print(dummyacc)
    # # Train classifier
    # clf = RandomForestClassifier(n_estimators=100, random_state=42)
    # clf.fit(X_train, y_train[:,i])
    # # Evaluate
    # y_pred = clf.predict(X_test)
    # print("Classifier")
    # print(classification_report(y_test[:,i], y_pred))

#####################
cane_height_type
#####################
Dummy
0.991704889235874
#####################
closure_placement
#####################
Dummy
0.5370352298233629
#####################
heel_shape_type
#####################
Dummy
0.9628509157151687
#####################
knit_structure
#####################
Dummy
0.8443772160957679
#####################
length_type
#####################
Dummy
0.8419862724049315
#####################
neck_lapel_type
#####################
Dummy
0.6515077583683029
#####################
silhouette_type
#####################
Dummy
0.8590969714713249
#####################
sleeve_length_type
#####################
Dummy
0.6436355356039166
#####################
toecap_type
#####################
Dummy
0.9662177547900198
#####################
waist_type
#####################
Dummy
0.7934680068963273
#####################
woven_structure
#####################
Dummy
0.65110113529163


Dummy
0.9912173700902659


In [38]:
max(1-dummyacc, dummyacc)

np.float64(0.9912173700902659)